In [1]:
import glob
import os
import pandas as pd
from pathlib import PosixPath

In [51]:
class ImagePath(PosixPath):
    
    """ Extends pathlib.PosixPath to parse object type and score.  
    May need some tweaking or may just work in Windows by 
    replacing PosixPath with WindowsPath. Not sure"""
    
#     def __init__(self, raw:str):
#         self.path = Path(raw)

    def is_valid(self):
        try:
            self.score()
            self.objtype()
            return True
        except:
            return False
    
    def score(self):
        return int(self.stem.split("-")[-2])
    
    def objtype(self):
        return self.stem.split("-")[-3]
    
    def idx(self):
        return int(self.stem.split("-")[-1])


In [52]:
#NBMODULE_IGNORE 
fp = ImagePath("/Users/jake/Code/cars/ruscha-masks/gri_2012_m_1_fn3_1975aug03_r001_0000a-car-99-0.png")
assert fp.score() == 99
assert fp.objtype() == "car"
assert fp.idx() == 0

In [55]:
class PathFormer:
    
    file_ext = ".png"
    
    
    def setattr_type(self, name, value, required_type):
        assert type(value) == required_type
        setattr(self, name, value)
        return self
        
    def setattr_str(self, name, value):
        return self.setattr_type(name, value, str)
        
    def setattr_int(self, name, value):
        return self.setattr_type(name, value, int)
        
    def set_parent(self, dir_path:str):
        return self.setattr_str("parent", dir_path)
        
    def set_file_stem(self, file_stem:str):
        return self.setattr_str("file_stem", file_stem)
        
    def set_file_ext(self, file_ext:str):
        return self.setattr_str("file_ext", file_ext)
    
    def set_score(self, score:int):
        return self.setattr_int("score", score)
    
    def set_objtype(self, objtype:str):
        return self.setattr_str("objtype", objtype)
    
    def set_idx(self, idx:int):
        return self.setattr_int("idx", idx)
    
    def path(self):
        return ImagePath(str(self))
        
    
    def __str__(self):
        return os.path.join(self.parent, f"{self.file_stem}-{self.objtype}-{self.score}-{self.idx}{self.file_ext}")
    

In [56]:
#NBMODULE_IGNORE    
pf = PathFormer()\
    .set_parent("../sample-images/masks/")\
    .set_file_stem("gri_2012_m_1_fn3_1975aug03_r001_0000a")\
    .set_objtype("car")\
    .set_score(99)\
    .set_idx(0)
#     .set_file_ext(".png")

pf.path()

ImagePath('../sample-images/masks/gri_2012_m_1_fn3_1975aug03_r001_0000a-car-99-0.png')

In [84]:
def picture_frame(dir_path, exts=["png", "jpg"], img_col="localpath"):
    
    """ Get a dataframe of image filenames from a directory of images with expected filename format"""
    
    if not os.path.isdir(dir_path):
        raise Exception(f"Invalid directory: {dir_path}")
        
    ret = pd.DataFrame({img_col:glob.glob(os.path.join(dir_path,"*.*"))})

    def valid_fname(fname):
        for ext in exts:
            if fname.lower().strip().endswith(ext.lower()):
                return True
        return False 
    
    ret = ret[ret[img_col].apply(valid_fname)]
    
    return ret

    ret = ret[ret[img_col].apply(lambda x: ImagePath(x).is_valid())]
    
    ret["score"] = ret[img_col].apply(lambda x: ImagePath(x).score())
    ret["objtype"] = ret[img_col].apply(lambda x: ImagePath(x).objtype())
    ret["idx"] = ret[img_col].apply(lambda x: ImagePath(x).idx())
    
    return ret


In [85]:
#NBMODULE_IGNORE
picture_frame("../sample-images/cropped/", exts=["png"])

,localpath
0,../sample-images/cropped/gri_2012_m_1_fn3_1975...
1,../sample-images/cropped/gri_2012_m_1_fn3_1975...
2,../sample-images/cropped/gri_2012_m_1_fn3_1975...
3,../sample-images/cropped/gri_2012_m_1_fn3_1975...
4,../sample-images/cropped/gri_2012_m_1_fn3_1975...
...,...
7675,../sample-images/cropped/gri_2012_m_1_fn3_1975...
7676,../sample-images/cropped/gri_2012_m_1_fn3_1975...
7677,../sample-images/cropped/gri_2012_m_1_fn3_1975...
7678,../sample-images/cropped/gri_2012_m_1_fn3_1975...
